In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import lxml
from IPython.core.display import display,HTML
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import lxml
from IPython.core.display import display,HTML
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#https://pypi.org/project/python-craigslist/

In [ ]:
civic_url = 'https://orangecounty.craigslist.org/d/cars-trucks-by-owner/search/cto?auto_make_model=civic&auto_title_status=1&max_auto_miles=100000&max_auto_year=2019&max_price=10000&min_auto_miles=30000&min_auto_year=2009&min_price=3000'
camry = ''
car_urls = []
kbb_base_url = 'https://www.kbb.com/honda/civic/YYYY/lx-sedan-4d/?category=sedan&condition=good&intent=buy-used&mileage=MMMMM&pricetype=private-party'

In [ ]:
response = requests.get(civic_url)

In [ ]:
#200 = Good; 400 = Bad Request, 403 = Forbidden, 404 Not Found
print('Status Code:', response.status_code)

In [ ]:
html = response.text

In [ ]:
soup = BeautifulSoup(html, features = 'lxml')

In [ ]:
soup.html.title

In [ ]:
result_list = soup.find_all('a', {'class': 'result-title hdrlnk'},href = True)

In [ ]:
result_list

# Make DF

In [ ]:
df = pd.DataFrame(index=range(len(result_list)),columns=['Photo','Model','Mileage','Asking','KBB_low','KBB_high','URL'])

In [ ]:
df

# Collect Relevant Ads

In [ ]:
posting_urls = [result_list[i]['href'] for i in range(len(result_list))]
#posting_urls = posting_urls[0:2]
posting_urls

In [ ]:
#Loop through all relevant posts and find 
for i,u in enumerate(posting_urls):
    
    print(i)
    print(u)
    r = requests.get(u)
    #print('Status Code:', response.status_code)
    h = r.text    
    s = BeautifulSoup(h, 'lxml')

    #Model:
    attrs = s.find_all('p', attrs={'class': 'attrgroup'})
    Model = attrs[0].find('span')
    Model = Model.text
    
    
    #Year:
    start = Model.find('20')
    Year = Model[start:start+4]
    print(Year)
    
    #print(Model)
    
    #Odometer
    a = attrs[1].find_all('span')
    a = [i.text for i in a]
    print(a)
    for j in a:
        if j.find('odometer: ')==0:
            Mileage = j.split('odometer: ',1)[1]
    print(Mileage)
            
    #Asking
    price = s.find_all('span', attrs={'class': 'price'})
    Asking = re.sub('[$,]', '',price[0].text )
    
    #Photo
    try:
        Photo = s.findAll('img')[0]['src']
    except:
        Photo = 'NOT FOUND'
    
    #KBB Avg Private Party Value
    kbb_url = kbb_base_url.replace('YYYY', str(Year),1)
    kbb_url = kbb_url.replace('MMMMM',Mileage)
    
    try:
        driver = webdriver.Chrome()
        driver.get(kbb_url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        t = soup.find('img',{'alt':'price advisor'})
        #t.attrs['src']
        resp = requests.get(t.attrs['src'])
        soup = BeautifulSoup(resp.content, 'html.parser')
        option_tags = soup.find_all("text",{'text-anchor':'middle'})
        option_tags
        price_range = option_tags[1].contents[0]
        low, high = price_range.split('-')
        KBB_low = re.sub('[^A-Za-z0-9]+', '', low)
        KBB_high = re.sub('[^A-Za-z0-9]+', '', high)
    except:
        print("#########################\n\n\n KBB ERROR \n\n\n#########################")
        KBB_low = float("NaN")
        KBB_high = float("NaN")
        driver.quit()



    
    #Set the data in df colums 'Photo','Model','Milage','Asking','KBB','URL'
    df.loc[i,'Model']      = Model
    df.loc[i,'Mileage']    = Mileage
    df.loc[i,'Asking']     = Asking
    df.loc[i,'URL']        = u
    df.loc[i,'Photo']      = Photo
    df.loc[i,'KBB_low']    = KBB_low     #if on low end of range, then include, otherwise set as NaN and remove from pd immediately after
    df.loc[i,'KBB_high']   = KBB_high     
    

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.loc[:,'Asking'] = df.loc[:,'Asking'].astype(int)
df.loc[:,'KBB_low']= df.loc[:,'KBB_low'].astype(int)
df.loc[:,'KBB_high']= df.loc[:,'KBB_high'].astype(int)
df.loc[:,'Mileage'] = df.loc[:,'Mileage'].astype('string')
df.loc[:,'Model'] = df.loc[:,'Model'].astype('string')

In [ ]:
df.dtypes

In [ ]:

df = df.sort_values(by = 'Asking', ascending = True)

In [ ]:
df = df.loc[df['Asking'] < df['KBB_low'],:] #If it's not below the KBB low estimation, just exclude it from the df

In [ ]:
df['offer_ratio'] = df['Asking'].div(df.KBB_low, axis = 0)

In [71]:
df = df.sort_values(by = 'offer_ratio', ascending = True)

# Find Convert image for Pandas Dataframe

In [ ]:
#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe/53469293

In [73]:
# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="180" >'

pd.set_option('display.max_colwidth', None)

# Create the dictionariy to be passed as formatters
format_dict = {}
format_dict['Photo'] = path_to_image_html


display(HTML(df.to_html(escape=False ,formatters=format_dict)))

,Photo,Model,Mileage,Asking,KBB_low,KBB_high,URL,offer_ratio
2,,2015 honda civic,42500,8600,13342,15535,https://losangeles.craigslist.org/wst/cto/d/long-beach-2015-honda-civic/7368820862.html,0.553589
4,,2010 honda civic hybrid,99615,6500,5244,7217,https://losangeles.craigslist.org/sgv/cto/d/hacienda-heights-2010-honda-civic-hybrid/7365850421.html,0.900651
5,,2012 honda civic,84500,9400,7824,10181,https://losangeles.craigslist.org/sfv/cto/d/encino-honda-civic/7359190369.html,0.923288
0,,2010 honda civic,99000,7000,5244,7217,https://losangeles.craigslist.org/sfv/cto/d/canyon-country-2010-honda-civic-coupe/7371136569.html,0.969932
3,,2012 Honda Civic Ex Coupe,89758,9600,7527,9884,https://losangeles.craigslist.org/sfv/cto/d/canoga-park-2012-honda-civic-ex-coupe/7366133129.html,0.971267
1,,2012 honda civic coupe ex,89700,9800,7527,9884,https://losangeles.craigslist.org/sfv/cto/d/tarzana-honda-2012-looks-and-drives/7369131724.html,0.991501


In [ ]:
d = HTML(df.to_html(escape=False ,formatters=format_dict))

# Email Relevant Adds

In [ ]:
#https://stackoverflow.com/questions/64505/sending-mail-from-python-using-smtp
#https://stackoverflow.com/questions/50564407/pandas-send-email-containing-dataframe-as-a-visual-table/50566309
#https://realpython.com/python-send-email/
#https://www.code-learner.com/python-send-html-image-and-attachment-email-example/
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib,ssl
import sys

In [ ]:

port = 465  # For SSL
password = '' #input("Type your password and press enter: ")

# Create a secure SSL context
context = ssl.create_default_context()

with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
    server.login("", password) #Put sending email here. Make sure to allow smtp send through gmail) just google how to do so
    # TODO: Send email here
    recipients = ['',''] #Sends to both of us 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = "Car deals, sorted by offer ratio"
    msg['From'] = '' #sender email


    html = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(df.loc[df['Asking'] < df['KBB_low'],:].to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server.sendmail(msg['From'], emaillist , msg.as_string())
